In [1]:
# GUI
import cv2
import numpy as np

def adaptive_kmeans(image):
    # Convert image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur
    blur = cv2.GaussianBlur(gray, (5, 5), 0)

    # Apply adaptive thresholding
    thresh = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_MEAN_C,
                                   cv2.THRESH_BINARY_INV, 11, 2)

    # Apply morphological opening
    kernel = np.ones((3, 3), np.uint8)
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)

    # Apply distance transform
    dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)

    # Apply thresholding
    ret, segmented = cv2.threshold(dist_transform, 0.5 * dist_transform.max(),
                                    255, 0)

    # Convert back to uint8
    segmented = np.uint8(segmented)

    return segmented

def segment_image(h_min, s_min, v_min, h_max, s_max, v_max,image=None):
    # capture a frame from the camera
#     ret, frame = cap.read()
    if image is not None:
#         frame = cv2.imread('hand1.jpg')
        frame = cv2.imread('hand.jpg')
    # convert the frame from BGR to HSV
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # create a mask using the trackbar values
    lower = np.array([h_min, s_min, v_min])
    upper = np.array([h_max, s_max, v_max])
    mask = cv2.inRange(hsv_frame, lower, upper)
    
    # Apply adaptive k-means algorithm for segmentation
#     segmented = adaptive_kmeans(image)

    # Combine the two masks
#     final_mask = cv2.bitwise_or(mask, segmented)

    # apply the mask to the original frame
#     result = cv2.bitwise_and(frame, frame, mask=mask)
    result = cv2.bitwise_and(frame, frame, mask=mask)

    # display the result
    cv2.imshow('Segmentation Result', result)
    


def hand_segmentation(image):
    # Convert image to HSV color space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Define range of skin color in HSV
    lower_skin = np.array([0, 20, 70], dtype=np.uint8)
    upper_skin = np.array([20, 255, 255], dtype=np.uint8)

    # Threshold the image to get only skin color
    mask = cv2.inRange(hsv, lower_skin, upper_skin)

    # Apply adaptive k-means algorithm for segmentation
    segmented = adaptive_kmeans(image)

    # Combine the two masks
    final_mask = cv2.bitwise_or(mask, segmented)

    # Apply morphological closing to remove noise
    kernel = np.ones((5, 5), np.uint8)
    closing = cv2.morphologyEx(final_mask, cv2.MORPH_CLOSE, kernel)

    # Apply Gaussian blur to smooth edges
    blur = cv2.GaussianBlur(closing, (7, 7), 0)

    return blur
    
    
cv2.namedWindow('Trackbars')
cv2.createTrackbar('H Min', 'Trackbars', 0, 179, lambda x: x)
cv2.createTrackbar('S Min', 'Trackbars', 0, 255, lambda x: x)
cv2.createTrackbar('V Min', 'Trackbars', 0, 255, lambda x: x)
cv2.createTrackbar('H Max', 'Trackbars', 179, 179, lambda x: x)
cv2.createTrackbar('S Max', 'Trackbars', 255, 255, lambda x: x)
cv2.createTrackbar('V Max', 'Trackbars', 255, 255, lambda x: x)
cap = cv2.VideoCapture(0)

while True:
    # get the trackbar values
    h_min = cv2.getTrackbarPos('H Min', 'Trackbars')
    s_min = cv2.getTrackbarPos('S Min', 'Trackbars')
    v_min = cv2.getTrackbarPos('V Min', 'Trackbars')
    h_max = cv2.getTrackbarPos('H Max', 'Trackbars')
    s_max = cv2.getTrackbarPos('S Max', 'Trackbars')
    v_max = cv2.getTrackbarPos('V Max', 'Trackbars')

    # segment the image and display the result
    segment_image(h_min, s_min, v_min, h_max, s_max, v_max, 1)
#     ret, frame = cap.read()
#     hand_segmentation(frame).
    

    # exit if the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window_w32.cpp:2576: error: (-27:Null pointer) NULL window: 'Trackbars' in function 'cvGetTrackbarPos'


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# 读取图像
img = cv2.imread('hand.jpg')

# 转换为HSV颜色空间
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# 根据HSV中的阈值初步定位手部区域
lower_skin = np.array([0, 45, 70], dtype=np.uint8)
upper_skin = np.array([20, 255, 255], dtype=np.uint8)
mask = cv2.inRange(hsv, lower_skin, upper_skin)

# 使用形态学运算去噪并移除定位区域中的孔
kernel = np.ones((5,5),np.uint8)
closing = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
opening = cv2.morphologyEx(closing, cv2.MORPH_OPEN, kernel)

# 在程序中显示提取的手部图片，并计算根据像素的肤色区域的平均RGB颜色值并输出结果。
result = cv2.bitwise_and(img,img,mask=opening)
mean_val = cv2.mean(result, mask=opening)
print(mean_val)
contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# 在原始图片中绘制轮廓
cv2.drawContours(img, contours, -1, (0, 255, 0), 3)
plt.imshow(img)
plt.show()
cv2.imshow(img)
cv2